In [1]:
import os
import imageio
import mobie
import mobie.htm as htm
import mobie.metadata as metadata

import ome_zarr

In [2]:
from ome_zarr.io import parse_url
import zarr

In [3]:
from os.path import join

In [4]:
from mobie.metadata.project_metadata import create_project_metadata, add_dataset
from mobie.metadata.dataset_metadata import create_dataset_structure, create_dataset_metadata
from mobie.metadata.source_metadata import add_source_to_dataset, add_regions_to_dataset

In [5]:
from mobie.view_utils import create_view
from mobie.metadata import get_default_view, add_view_to_dataset

In [6]:
plate_path = "./zarr-files/Projection-Mix.zarr/"
store = parse_url(plate_path, mode="w").store
    
plate = zarr.group(store=store)

In [7]:
mobie_project_folder = "./mobie-projects/Z-Stack"

In [8]:
dataset_name = "Projection-Mix"

In [9]:
create_project_metadata(mobie_project_folder)

In [10]:
create_dataset_structure(mobie_project_folder, dataset_name, file_formats=["ome.zarr"])

'./mobie-projects/Z-Stack/Projection-Mix'

In [11]:
create_dataset_metadata(join(mobie_project_folder, dataset_name), is2d=False)

In [12]:
add_dataset(mobie_project_folder, dataset_name, is_default=True)

In [13]:
from mobie.view_utils import create_grid_view

In [14]:
sources = {}
positions = {}
colors = {}
well_sources = {}
histograms = {}

In [15]:
import string
def to_position(well_name):
    r,c = well_name[0], well_name[1:]
    r = string.ascii_uppercase.index(r)
    c = int(c) - 1
    return [c, r]

In [16]:
def hex_to_rgba(h):
    if not isinstance(h, str):
        return "r=255,g=255,b=255,a=255"
    return f"r={int(h[0:2], 16)},g={int(h[2:4], 16)},b={int(h[4:6], 16)},a=255"

In [17]:
from mobie.view_utils import create_view

In [18]:
from faim_hcs.UIntHistogram import UIntHistogram

In [19]:
def get_max_field_yx(plate):
    max_y, max_x = 0, 0
    for row in plate.group_keys():
        for col in plate[row].group_keys():
            shape = plate[row][col][0][0].shape
            if max_y < shape[-2]:
                max_y = shape[-2]
            
            if max_x < shape[-1]:
                max_x = shape[-1]
                
    return max_y, max_x

In [20]:
get_max_field_yx(plate)

(512, 1024)

In [21]:
from copy import copy

In [22]:
gap = 50
y_max, x_max = get_max_field_yx(plate)

sources = {}
source_transforms = {}
display_settings = {}
plate_hists = {}

wells = []

for i, row in enumerate(plate.group_keys()):
    for j, col in enumerate(plate[row].group_keys()):
        attrs = plate[row][col]['0'].attrs.asdict()
        _, z_spacing, y_spacing, x_spacing = attrs['multiscales'][0]['datasets'][0]['coordinateTransformations'][0]['scale']
        wells.append(row + col.zfill(2))
        path = join(plate.store.path, row, col, '0')

        hists = [UIntHistogram.load(join(path, h_path)) for h_path in attrs['histograms']]
        
        for k, ch in enumerate(attrs['omero']['channels']):
            key = f"{ch['wavelength_id']}_{ch['label']}"
            
            name = f"{row}{col.zfill(2)}_{key}"
            source_transform = {
                    "sources": [name],
                    "parameters": [1., 0., 0., (x_max + gap) * x_spacing * j, 
                                   0., 1., 0., (y_max + gap) * y_spacing * i,
                                   0., 0., 1., 0.],
                }
            view = get_default_view(
                source_type="image",
                source_name=name,
                menu_name="Wells",
                source_transform=source_transform,
                color=hex_to_rgba(ch['color']),
                contrastLimits=[hists[k].quantile(0.01), hists[k].quantile(0.99)],
                opacity=1.0,
                sources=[
                    name,
                ]
            )            

            add_source_to_dataset(
                dataset_folder=join(mobie_project_folder, dataset_name),
                source_type="image",
                source_name=name,
                image_metadata_path=path,
                file_format="ome.zarr",
                channel=k,
                view=view
            )
            
            if key not in plate_hists.keys():
                plate_hists[key] = copy(hists[k])
            else:
                plate_hists[key].combine(hists[k])
            
            if key not in sources.keys():
                sources[key] = [name]
            else:
                sources[key].append(name)
                
            if key not in source_transforms.keys():
                source_transforms[key] = [{"affine": source_transform}]
            else:
                source_transforms[key].append({"affine": source_transform})
                
            if key not in display_settings.keys():
                display_settings[key] = {
                        "color": hex_to_rgba(ch['color']),
                        "opacity": 1.0,
                        "sources": [
                            name,
                        ],
                    }
                
            else:
                display_settings[key]["sources"].append(name)

In [23]:
from mobie.metadata import get_grid_view, get_merged_grid_source_transform, get_image_display

In [24]:
from mobie.view_utils import create_view

In [25]:
first_key = list(source_transforms.keys())[0]
src_trafos = source_transforms[first_key]
for c in filter(lambda k: k != first_key, source_transforms.keys()):
    for i, trafo in enumerate(source_transforms[c]):
        src_trafos[i]['affine']['sources'].extend(trafo['affine']['sources'])

In [26]:
disp_settings = []
group_names = []
sources_ = []

for ch in display_settings.keys():
    ds = display_settings[ch]
    ds['contrastLimits'] = [plate_hists[ch].quantile(0.01), plate_hists[ch].quantile(0.99)]
    disp_settings.append(ds)
    
    group_names.append(ch)
    
    sources_.append(sources[ch])
    
    create_view(
        dataset_folder=join(mobie_project_folder, dataset_name),
        view_name=ch,
        display_settings=[ds],
        display_group_names=[ch],
        sources=[sources[ch]],
        source_transforms=source_transforms[ch],
        menu_name="Channels",
        is_exclusive=False,
    )

In [27]:
disp_settings

[{'color': 'r=115,g=255,b=0,a=255',
  'opacity': 1.0,
  'contrastLimits': [111, 8065]},
 {'color': 'r=115,g=255,b=0,a=255',
  'opacity': 1.0,
  'contrastLimits': [111, 8014]},
 {'color': 'r=0,g=0,b=0,a=255', 'opacity': 1.0, 'contrastLimits': [0, 0]},
 {'color': 'r=115,g=255,b=0,a=255',
  'opacity': 1.0,
  'contrastLimits': [0, 170]}]

In [28]:
import pandas as pd

In [29]:
well_table = pd.DataFrame({"region_id": wells,
                          "treatment": ["Unknown",]*len(wells)})

In [30]:
well_table

,region_id,treatment
0,E07,Unknown
1,E08,Unknown


In [31]:
add_regions_to_dataset(join(mobie_project_folder, dataset_name), "wells", well_table)

In [32]:
from mobie.metadata import get_region_display

In [33]:
sources

{'C01_FITC_05': ['E07_C01_FITC_05', 'E08_C01_FITC_05'],
 'C02_FITC_05': ['E07_C02_FITC_05', 'E08_C02_FITC_05'],
 'C03_empty': ['E07_C03_empty', 'E08_C03_empty'],
 'C04_FITC_05': ['E07_C04_FITC_05', 'E08_C04_FITC_05']}

In [34]:
well_sources = {}

for ch in sources.keys():    
    for well in sources[ch]:
        if well[:3] not in well_sources.keys():
            well_sources[well[:3]] = [well]
        else:
            well_sources[well[:3]].append(well)

In [35]:
well_sources

{'E07': ['E07_C01_FITC_05',
  'E07_C02_FITC_05',
  'E07_C03_empty',
  'E07_C04_FITC_05'],
 'E08': ['E08_C01_FITC_05',
  'E08_C02_FITC_05',
  'E08_C03_empty',
  'E08_C04_FITC_05']}

In [36]:
region_disp = get_region_display("wells", well_sources, table_source="wells", lut="glasbey", opacity=0.5, visible=True)

In [37]:
region_disp

{'regionDisplay': {'opacity': 0.5,
  'lut': 'glasbey',
  'name': 'wells',
  'sources': {'E07': ['E07_C01_FITC_05',
    'E07_C02_FITC_05',
    'E07_C03_empty',
    'E07_C04_FITC_05'],
   'E08': ['E08_C01_FITC_05',
    'E08_C02_FITC_05',
    'E08_C03_empty',
    'E08_C04_FITC_05']},
  'tableSource': 'wells',
  'visible': True}}

In [38]:
create_view(
    dataset_folder=join(mobie_project_folder, dataset_name),
    view_name="default",
    display_settings=disp_settings,
    display_group_names=group_names,
    sources=sources_,
    source_transforms=src_trafos,
    region_displays={
        'Wells': {
            'opacity': 0.5,
            'lut': 'glasbey',
            'sources': well_sources,
            'tableSource': 'wells',
            'visible': True,
            'showAsBoundaries': True,
        }
    },
    menu_name="Overview",
)

In [39]:
mobie.validation.validate_project(mobie_project_folder)

Check views for dataset Projection-Mix: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 71.85it/s]
Check view files for dataset Projection-Mix: 0it [00:00, ?it/s]

The project at ./mobie-projects/Z-Stack is a valid MoBIE project.


# Add Projections

In [42]:
# Same as above for stacks
# gap = 50
# y_max, x_max = get_max_field_yx(plate)

sources = {}
source_transforms = {}
display_settings = {}
plate_hists = {}

wells = []

for i, row in enumerate(plate.group_keys()):
    for j, col in enumerate(plate[row].group_keys()):
        attrs = plate[row][col]['0']['projections'].attrs.asdict()
        _, y_spacing, x_spacing = attrs['multiscales'][0]['datasets'][0]['coordinateTransformations'][0]['scale']
        wells.append(row + col.zfill(2))
        path = join(plate.store.path, row, col, '0', 'projections')

        hists = [UIntHistogram.load(join(path, h_path)) for h_path in attrs['histograms']]
        
        for k, ch in enumerate(attrs['omero']['channels']):
            key = f"{ch['wavelength_id']}_{ch['label']}"
            
            name = f"{row}{col.zfill(2)}_{key}"
            source_transform = {
                    "sources": [name],
                    "parameters": [1., 0., 0., (x_max + gap) * x_spacing * j, 
                                   0., 1., 0., (y_max + gap) * y_spacing * i,
                                   0., 0., 1., 0.],
                }
            view = get_default_view(
                source_type="image",
                source_name=name,
                menu_name="Projections",
                source_transform=source_transform,
                color=hex_to_rgba(ch['color']),
                contrastLimits=[hists[k].quantile(0.01), hists[k].quantile(0.99)],
                opacity=1.0,
                sources=[
                    name,
                ]
            )            

            add_source_to_dataset(
                dataset_folder=join(mobie_project_folder, dataset_name),
                source_type="image",
                source_name=name,
                image_metadata_path=path,
                file_format="ome.zarr",
                channel=k,
                view=view
            )
            
            if key not in plate_hists.keys():
                plate_hists[key] = copy(hists[k])
            else:
                plate_hists[key].combine(hists[k])
            
            if key not in sources.keys():
                sources[key] = [name]
            else:
                sources[key].append(name)
                
            if key not in source_transforms.keys():
                source_transforms[key] = [{"affine": source_transform}]
            else:
                source_transforms[key].append({"affine": source_transform})
                
            if key not in display_settings.keys():
                display_settings[key] = {
                        "color": hex_to_rgba(ch['color']),
                        "opacity": 1.0,
                        "sources": [
                            name,
                        ],
                    }
                
            else:
                display_settings[key]["sources"].append(name)

In [43]:
from mobie.metadata import get_grid_view, get_merged_grid_source_transform, get_image_display

In [44]:
from mobie.view_utils import create_view

In [45]:
first_key = list(source_transforms.keys())[0]
src_trafos = source_transforms[first_key]
for c in filter(lambda k: k != first_key, source_transforms.keys()):
    for i, trafo in enumerate(source_transforms[c]):
        src_trafos[i]['affine']['sources'].extend(trafo['affine']['sources'])

In [46]:
src_trafos

[{'affine': {'sources': ['E07_C01_Maximum-Projection_FITC_05',
    'E07_C02_Best-Focus-Projection_FITC_05',
    'E07_C03_Maximum-Projection_FITC_05',
    'E07_C04_empty'],
   'parameters': [1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    1.0,
    0.0]}},
 {'affine': {'sources': ['E08_C01_Maximum-Projection_FITC_05',
    'E08_C02_Best-Focus-Projection_FITC_05',
    'E08_C03_Maximum-Projection_FITC_05',
    'E08_C04_empty'],
   'parameters': [1.0,
    0.0,
    0.0,
    1467.9432,
    0.0,
    1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    1.0,
    0.0]}}]

In [47]:
list(display_settings.keys())

['C01_Maximum-Projection_FITC_05',
 'C02_Best-Focus-Projection_FITC_05',
 'C03_Maximum-Projection_FITC_05',
 'C04_empty']

In [48]:
disp_settings = []
group_names = []
sources_ = []

for ch in display_settings.keys():
    ds = display_settings[ch]
    ds['contrastLimits'] = [plate_hists[ch].quantile(0.01), plate_hists[ch].quantile(0.99)]
    disp_settings.append(ds)
    
    group_names.append(ch)
    
    sources_.append(sources[ch])
    
    create_view(
        dataset_folder=join(mobie_project_folder, dataset_name),
        view_name=ch,
        display_settings=[ds],
        display_group_names=[ch],
        sources=[sources[ch]],
        source_transforms=source_transforms[ch],
        menu_name="Channels",
        is_exclusive=False,
    )

In [49]:
sources

{'C01_Maximum-Projection_FITC_05': ['E07_C01_Maximum-Projection_FITC_05',
  'E08_C01_Maximum-Projection_FITC_05'],
 'C02_Best-Focus-Projection_FITC_05': ['E07_C02_Best-Focus-Projection_FITC_05',
  'E08_C02_Best-Focus-Projection_FITC_05'],
 'C03_Maximum-Projection_FITC_05': ['E07_C03_Maximum-Projection_FITC_05',
  'E08_C03_Maximum-Projection_FITC_05'],
 'C04_empty': ['E07_C04_empty', 'E08_C04_empty']}

In [50]:
well_sources = {}

for ch in sources.keys():    
    for well in sources[ch]:
        if well[:3] not in well_sources.keys():
            well_sources[well[:3]] = [well]
        else:
            well_sources[well[:3]].append(well)

In [51]:
well_sources

{'E07': ['E07_C01_Maximum-Projection_FITC_05',
  'E07_C02_Best-Focus-Projection_FITC_05',
  'E07_C03_Maximum-Projection_FITC_05',
  'E07_C04_empty'],
 'E08': ['E08_C01_Maximum-Projection_FITC_05',
  'E08_C02_Best-Focus-Projection_FITC_05',
  'E08_C03_Maximum-Projection_FITC_05',
  'E08_C04_empty']}

In [52]:
region_disp = get_region_display("wells", well_sources, table_source="wells", lut="glasbey", opacity=0.5, visible=True)

In [53]:
region_disp

{'regionDisplay': {'opacity': 0.5,
  'lut': 'glasbey',
  'name': 'wells',
  'sources': {'E07': ['E07_C01_Maximum-Projection_FITC_05',
    'E07_C02_Best-Focus-Projection_FITC_05',
    'E07_C03_Maximum-Projection_FITC_05',
    'E07_C04_empty'],
   'E08': ['E08_C01_Maximum-Projection_FITC_05',
    'E08_C02_Best-Focus-Projection_FITC_05',
    'E08_C03_Maximum-Projection_FITC_05',
    'E08_C04_empty']},
  'tableSource': 'wells',
  'visible': True}}

In [54]:
create_view(
    dataset_folder=join(mobie_project_folder, dataset_name),
    view_name="Projections",
    display_settings=disp_settings,
    display_group_names=group_names,
    sources=sources_,
    source_transforms=src_trafos,
    region_displays={
        'Wells': {
            'opacity': 0.5,
            'lut': 'glasbey',
            'sources': well_sources,
            'tableSource': 'wells',
            'visible': True,
            'showAsBoundaries': True,
        }
    },
    menu_name="Overview",
)

In [55]:
mobie.validation.validate_project(mobie_project_folder)

Check views for dataset Projection-Mix: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 67.81it/s]
Check view files for dataset Projection-Mix: 0it [00:00, ?it/s]

The project at ./mobie-projects/Z-Stack is a valid MoBIE project.
